In [192]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [193]:
# creating dataframe
df = pd.DataFrame([[8, 8, 4], [7, 9, 5], [6, 10, 6], [5, 12, 7]], columns=['cgpa', 'profile_score', 'lpa'])

In [194]:
df

,cgpa,profile_score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,12,7


In [195]:
# initialise parameters --> weights and bias

def initialise_paramters(layer_dimension):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dimension)

    for l in range(1, L):
        parameters['W' + str(l)] = np.ones((layer_dimension[l-1], layer_dimension[l])) * 0.1
        parameters['b' + str(l)] = np.zeros((layer_dimension[l], 1))
    return parameters    

In [196]:
dim = [2, 2, 1]
print(initialise_paramters(dim))


{'W1': array([[0.1, 0.1],
       [0.1, 0.1]]), 'b1': array([[0.],
       [0.]]), 'W2': array([[0.1],
       [0.1]]), 'b2': array([[0.]])}


In [197]:
# prediction/result after forward propogate
# activation is linear only 

def forward_propogation_linear(prev_nodes, W, b):
    Z = np.dot(W.T, prev_nodes) + b
    return Z

In [198]:
# forward propogation

def forward_propogation(X, paramters):
    A = X
    L = len(paramters)//2

    for l in range(1, L+1):
        A_prev = A
        Wl = paramters['W'+str(l)]
        bl = paramters['b'+str(l)]

        A = forward_propogation_linear(A_prev, Wl, bl)
    return A, A_prev # return A_prev i.e. prev node values because we will need them in weight updation

In [199]:
# update parameters 

def update_parameters(parameters,y,y_hat,A1,X):
  parameters['W2'][0][0] = parameters['W2'][0][0] + (0.001 * 2 * (y - y_hat)*A1[0][0])
  parameters['W2'][1][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat)*A1[1][0])
  parameters['b2'][0][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat))

  parameters['W1'][0][0] = parameters['W1'][0][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0]*X[0][0])
  parameters['W1'][0][1] = parameters['W1'][0][1] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0]*X[1][0])
  parameters['b1'][0][0] = parameters['b1'][0][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0])

  parameters['W1'][1][0] = parameters['W1'][1][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0]*X[0][0])
  parameters['W1'][1][1] = parameters['W1'][1][1] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0]*X[1][0])
  parameters['b1'][1][0] = parameters['b1'][1][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0])

In [200]:
# fitting 

parameters = initialise_paramters(dim)
epochs = 5

for i in range(epochs):

  Loss = []

  for j in range(df.shape[0]):
    
    X = df[['cgpa', 'profile_score']].values[j].reshape(2,1) # Shape(no of features, no. of training example)
    y = df[['lpa']].values[j][0]
    
    #  forward propogation 
    y_hat,A1 = forward_propogation(X,parameters)
    y_hat = y_hat[0][0]

    update_parameters(parameters,y,y_hat,A1,X)

    Loss.append((y-y_hat)**2)

  print('Epoch - ',i+1,'Loss - ',np.array(Loss).mean())

parameters

Epoch -  1 Loss -  25.321744156025517
Epoch -  2 Loss -  18.320004165722047
Epoch -  3 Loss -  9.473661050729625
Epoch -  4 Loss -  3.252093863403161
Epoch -  5 Loss -  1.3407132589299964


{'W1': array([[0.26507636, 0.38558861],
        [0.27800387, 0.40980287]]),
 'b1': array([[0.02749056],
        [0.02974394]]),
 'W2': array([[0.41165744],
        [0.48302736]]),
 'b2': array([[0.48646246]])}

## Same thing using Keras


In [201]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [202]:
# initialising our model 

model = Sequential()
model.add(Dense(2, activation='linear', input_dim=2))
model.add(Dense(1, activation='linear'))


In [203]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                     │ (None, 2)                   │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

In [204]:
# weight are ramdomly generated 
model.get_weights()

[array([[ 0.39301968, -0.2461369 ],
        [ 0.9646269 ,  0.19534469]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[ 0.5880693],
        [-0.6726495]], dtype=float32),
 array([0.], dtype=float32)]

In [205]:
# we will set the same weights and bais that we used above

new_weights = [np.array([[0.1, 0.1],
       [0.1, 0.1]], dtype=np.float32),
 np.array([0., 0.], dtype=np.float32),
 np.array([[0.1],
        [0.1]], dtype=np.float32),
 np.array([0.], dtype=np.float32)]

In [206]:
# setting new weights

model.set_weights(new_weights)

In [207]:
# setting up optimizer 
optimiser = Adam(learning_rate=0.001)

In [208]:
# compiling model

model.compile(optimizer=optimiser, loss='mean_squared_error')

In [209]:
t# converting into arrays

X = df[['cgpa', 'profile_score']].values
Y = df['lpa'].values

In [210]:
# fitting model

model.fit(X, Y, epochs=100, verbose=1, batch_size=1)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 25.6371
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27.1790 
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24.3404 
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.1183  
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 30.3852 
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.1297 
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26.1200 
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.5355 
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 22.7572 
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.6113 
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.6807 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 22.7482 
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 20.2867 
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 19.0753 
Epoch 15/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 20.5367
Epoch